In [2]:
movies_order_test = copy.deepcopy(movies_order_svd)
overall_average_test = overall_average_svd 
cnt_test = cnt_svd
test_rating_to_predict = []

for user in user_to_data_test:
    rand_num  = random.randint(0, len(user)-1)
    index = 0
    user_rating_sum = 0
    usr_rating_count = 0


    #These three are used to build feature_2_test
    words = OrderedSet()
    list_of_movie_to_count_dict = []
    target_word_counts = dict()
    non_target_ratings = []

    for movie in user:

        movies_order_test.add(movie[1])

        #corpus extraction:
        movie_string = ""
        for j in range (3,len(movie)):
            if(j!= len(movie)-1):
                movie_string+= movie[j]+" "
            else:
                movie_string+= movie[j]

        #LOOK: this declaration may need to be outside the loop
        wnl = WordNetLemmatizer()
        cleaned_string = remove_stopwords(movie_string)
        cleaned_list = [wnl.lemmatize(word) for word in cleaned_string.split(" ")]
        cleaned_list = [word[:-1] for word in cleaned_list if word.endswith(".")] + [word for word in cleaned_list if not word.endswith(".")]


        #For each user there needs to be an ordered set of words/terms in movies...
        words.update(cleaned_list)

        if(index == rand_num):
            test_rating_to_predict.append([int(movie[0]), int(movie[1])])
            target_rating_test.append(float(movie[2]))

            #for every movie there needs to be a dictionary of words to counts
            target_word_counts = Counter(cleaned_list)
        else:
            overall_average_test+=float(movie[2])
            cnt_test += 1
            user_rating_sum+=float(movie[2])
            usr_rating_count +=1
            test_list.append([int(movie[0]), int(movie[1]), float(movie[2])])

            #for every movie there needs to be a dictionary of words to counts
            list_of_movie_to_count_dict.append(Counter(cleaned_list))
            non_target_ratings.append(float(movie[2]))

        index+=1

    #LOOK: Now predict with average ratings:
    if(user_rating_sum==0):
        feature_1_test.append(-1)
    else: 
        feature_1_test.append(user_rating_sum/usr_rating_count)

    #LOOK: now predict with weighted rating...
    #LOOK: May want to use list comprehension here...
    complete_word_counts_in_order = []
    target_word_counts_in_order = []

    for temp_dict in list_of_movie_to_count_dict:
        word_count = []
        for movie in words:
            if movie in temp_dict.keys():
                word_count.append(temp_dict[movie])
            else:
                word_count.append(0)
        complete_word_counts_in_order.append(word_count)

    for movie in words:
        if movie in target_word_counts.keys():
            target_word_counts_in_order.append(target_word_counts[movie])
        else:
            target_word_counts_in_order.append(0)


    complete_word_counts = complete_word_counts_in_order.copy()
    complete_word_counts.append(target_word_counts_in_order)
    transformed_word_counts = TfidfTransformer().fit_transform(complete_word_counts).toarray()


    cosine_sim = linear_kernel(X = transformed_word_counts[0:-1],Y = [transformed_word_counts[-1]])
    cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))
    numerator = 0
    denominator = 0


    #LOOK: are cosine_sim and non_target_ratings in the same order???(yes)
    for i in range(len(non_target_ratings)):
        numerator += cosine_sim[i]*non_target_ratings[i]
        denominator += cosine_sim[i]

    # populate feature_2_test handeling fringe cases
    if denominator == 0:
        if(feature_1_test[-1] == -1):
            if(overall_average_test == 0):
                feature_2_test.append(3)
            else:
                feature_2_test.append(overall_average_test)
        else:
            feature_2_test.append(feature_1_test[-1])
    else:
        feature_2_test.append(numerator/denominator)



overall_average_test = overall_average_test/cnt_test

for i in range(len(feature_1_test)):
    if feature_1_test[i] ==-1:
        feature_1_test[i] = overall_average_test

random.shuffle(test_list)

1
